In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras import backend as K
K.clear_session()
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model

Using TensorFlow backend.


In [2]:
from keras.layers import Conv2D, MaxPool2D

input_shape = (32, 32, 3) 

model = Sequential(name='vgg16-sequential')

model.add(Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=input_shape, name='block1_conv1'))
model.add(Conv2D(16, (3, 3), padding='same', activation='relu', name='block1_conv2'))
model.add(MaxPool2D((2, 2), strides=(2, 2), name='block1_pool'))

model.add(Flatten(name='flatten'))

model.add(Dense(50, activation='softmax', name='predictions'))

model.summary()

Model: "vgg16-sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 16)        448       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
predictions (Dense)          (None, 50)                204850    
Total params: 207,618
Trainable params: 207,618
Non-trainable params: 0
_________________________________________________________________


In [3]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [10]:
train_path = "Training_new"
test_path = "Test_32x32"
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(32, 32), class_mode="categorical", batch_size=251)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(32, 32), class_mode="categorical", batch_size=157)

Found 25100 images belonging to 50 classes.
Found 8478 images belonging to 50 classes.


In [11]:
model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(train_batches, steps_per_epoch=100,validation_data=test_batches,validation_steps=54,epochs=11,verbose=2)

Epoch 1/11
 - 25s - loss: 9.0018 - accuracy: 0.4816 - val_loss: 0.9083 - val_accuracy: 0.7876
Epoch 2/11
 - 11s - loss: 0.0552 - accuracy: 0.9871 - val_loss: 0.3702 - val_accuracy: 0.8950
Epoch 3/11
 - 11s - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.3659 - val_accuracy: 0.9173
Epoch 4/11
 - 11s - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.4749 - val_accuracy: 0.9213
Epoch 5/11
 - 11s - loss: 6.1083e-04 - accuracy: 1.0000 - val_loss: 0.2699 - val_accuracy: 0.9276
Epoch 6/11
 - 11s - loss: 9.8019e-04 - accuracy: 0.9999 - val_loss: 0.4906 - val_accuracy: 0.9246
Epoch 7/11
 - 11s - loss: 4.7414e-04 - accuracy: 1.0000 - val_loss: 0.2814 - val_accuracy: 0.9306
Epoch 8/11
 - 11s - loss: 1.5586e-04 - accuracy: 1.0000 - val_loss: 0.3116 - val_accuracy: 0.9304
Epoch 9/11
 - 11s - loss: 1.0821e-04 - accuracy: 1.0000 - val_loss: 0.5696 - val_accuracy: 0.9336
Epoch 10/11
 - 11s - loss: 9.0676e-05 - accuracy: 1.0000 - val_loss: 0.4662 - val_accuracy: 0.9377
Epoch 11/11
 - 11s - loss: 7.1984e-

In [13]:
model.layers[4]

In [14]:
print("Biases : ", model.layers[4].get_weights()[1].shape)

Biases :  (50,)


In [239]:
model.layers[4].get_weights()[0]

array([[-0.02455233, -0.01121073, -0.00709064, ..., -0.01361853,
         0.03695786, -0.04038417],
       [-0.01235719, -0.02359619,  0.01707634, ...,  0.03856889,
         0.00445458,  0.03962469],
       [ 0.02684362, -0.01365526, -0.00883746, ..., -0.02740848,
         0.02116891,  0.04208881],
       ...,
       [ 0.02313991, -0.06889132, -0.02330975, ..., -0.03010493,
        -0.02544045,  0.019929  ],
       [-0.02159299, -0.02008313,  0.02679505, ..., -0.00425829,
        -0.00907113,  0.01336208],
       [ 0.00825535, -0.00033172,  0.01276093, ...,  0.03733187,
         0.03966337,  0.02495866]], dtype=float32)

In [141]:
pr=wt.flatten()

In [15]:
inputFeatures = model.layers[4].get_weights()[0].shape[0]
outFeatures = model.layers[4].get_weights()[0].shape[1]
print("Input features dim : ", inputFeatures)
print("Out Features dim : ", outFeatures)

Input features dim :  4096
Out Features dim :  50


In [144]:
np.savetxt('f.txt',pr,fmt='%f')

In [16]:
# Fully connected
for fc_layer in [4]:
  #fc_layer = 7   # 5,6,7
  inputFeatures = model.layers[fc_layer].get_weights()[0].shape[0]
  outFeatures = model.layers[fc_layer].get_weights()[0].shape[1]
  print("Input features dim : ", inputFeatures)
  print("Out Features dim : ", outFeatures)
  fileName = model.layers[fc_layer].name
  fileName += ".txt"
  fileHandler = open(fileName, "w")
  for outFeature in range(0, outFeatures):
    for inFeature in range(0, inputFeatures):
      fileHandler.write(str(model.layers[fc_layer].get_weights()[0][inFeature][outFeature]) + " ")
  fileHandler.write("\n")

  for bias in range(0, outFeatures):
    fileHandler.write(str(model.layers[fc_layer].get_weights()[1][bias]) + " ")
  fileHandler.close()

Input features dim :  4096
Out Features dim :  50


In [17]:
layer_no = 0 # 2,0
for layer_no in [0,1]:
  kernelWidth = model.layers[layer_no].get_weights()[0].shape[0]
  kernelHeight = model.layers[layer_no].get_weights()[0].shape[1]
  kernelDepth = model.layers[layer_no].get_weights()[0].shape[2]
  print("Kernel Width : ", kernelWidth)
  print("Kernel Height : ", kernelHeight)
  print("Kernel Depth : ", kernelDepth)
  numChannels = model.layers[layer_no].get_weights()[0].shape[3]
  print("Num Channels : ", numChannels)
  fileName = model.layers[layer_no].name
  fileName += ".txt"
  fileHandler = open(fileName, "w")
  for channels in range(0, numChannels):
    for depth in range(kernelDepth):
      for width in range(kernelWidth):
        for height in range(kernelHeight):
          fileHandler.write(str(model.layers[layer_no].get_weights()[0][width, height, depth, channels]) + " ")
  fileHandler.write("\n")
  # Store biases
  for channels in range(0, numChannels):
    fileHandler.write(str(model.layers[layer_no].get_weights()[1][channels]) + " ");
  fileHandler.close()

Kernel Width :  3
Kernel Height :  3
Kernel Depth :  3
Num Channels :  16
Kernel Width :  3
Kernel Height :  3
Kernel Depth :  16
Num Channels :  16


In [18]:
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Permute

layer1 = Input(shape=(32, 32, 3))
layer2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu',padding='same')(layer1)
layer3 = Conv2D(filters=16, kernel_size=(3, 3),padding='same', activation='relu')(layer2)
layer4 = MaxPooling2D(pool_size=(2, 2))(layer3)
layer5 = Flatten(data_format='channels_first')(layer4)
layer6 = Dense(50, activation='linear')(layer5)


# modelTrun = Model(inputs=layer1, outputs=layer2)

In [245]:
from keras.models import Model
modelTrun = Model(inputs=layer1, outputs=layer6)

In [246]:
modelTrun.layers[1].set_weights(model.layers[0].get_weights())
print(modelTrun.layers[1].name)
modelTrun.layers[2].set_weights(model.layers[1].get_weights())
print(modelTrun.layers[2].name)
modelTrun.layers[3].set_weights(model.layers[2].get_weights())
print(modelTrun.layers[3].name)
modelTrun.layers[4].set_weights(model.layers[3].get_weights())
print(modelTrun.layers[4].name)
modelTrun.layers[5].set_weights(model.layers[4].get_weights())
print(modelTrun.layers[5].name)


conv2d_1
conv2d_2
max_pooling2d_1
flatten_1
dense_1


In [247]:
from matplotlib.image import imread
img2 = imread("Test_32x32/Banana/12_100.jpg")
print(type(img2))

<class 'numpy.ndarray'>


In [230]:
img2


array([[[251, 255, 248],
        [251, 255, 250],
        [252, 255, 251],
        ...,
        [254, 255, 246],
        [255, 255, 253],
        [250, 248, 251]],

       [[251, 255, 250],
        [251, 255, 250],
        [252, 255, 251],
        ...,
        [251, 254, 245],
        [252, 252, 250],
        [255, 254, 255]],

       [[251, 255, 251],
        [252, 255, 251],
        [252, 255, 253],
        ...,
        [255, 255, 251],
        [250, 248, 249],
        [255, 253, 255]],

       ...,

       [[253, 255, 245],
        [253, 255, 245],
        [252, 255, 246],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[250, 254, 253],
        [250, 254, 253],
        [252, 255, 255],
        ...,
        [255, 255, 255],
        [255, 254, 255],
        [255, 254, 255]],

       [[251, 254, 255],
        [252, 255, 255],
        [252, 255, 255],
        ...,
        [255, 254, 255],
        [255, 254, 255],
        [255, 254, 255]]

In [248]:
img_reshaped=np.stack((img2[:,:,0],img2[:,:,1], img2[:,:,2]))

In [249]:
print(img_reshaped)

[[[251 251 252 ... 254 255 250]
  [251 251 252 ... 251 252 255]
  [251 252 252 ... 255 250 255]
  ...
  [253 253 252 ... 255 255 255]
  [250 250 252 ... 255 255 255]
  [251 252 252 ... 255 255 255]]

 [[255 255 255 ... 255 255 248]
  [255 255 255 ... 254 252 254]
  [255 255 255 ... 255 248 253]
  ...
  [255 255 255 ... 255 255 255]
  [254 254 255 ... 255 254 254]
  [254 255 255 ... 254 254 254]]

 [[248 250 251 ... 246 253 251]
  [250 250 251 ... 245 250 255]
  [251 251 253 ... 251 249 255]
  ...
  [245 245 246 ... 255 255 255]
  [253 253 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]]


In [199]:
img_fl = img_reshaped.flatten()
img_fl

array([251, 251, 252, ..., 255, 255, 255], dtype=uint8)

In [116]:
img_rs_fl = img2.flatten()
img_rs_fl

array([244, 250, 246, ..., 255, 255, 255], dtype=uint8)

In [260]:
out = modelTrun.predict(img2.reshape(1,32,32,3))

In [261]:
out



array([[ -0.07412484,  13.488444  , -30.044628  , -25.289286  ,
          1.9167101 , -15.434257  , -18.87718   , -11.921204  ,
         -3.9826117 ,   3.5211773 ,  -7.742969  ,   3.2652245 ,
        -13.832905  ,  -0.30899525,  -6.493058  , -11.331507  ,
         -6.666646  ,   7.3075337 ,  12.901896  , -19.338577  ,
         -6.324942  ,  -8.186701  ,  -7.810067  ,   5.4808464 ,
          1.5766498 ,  14.2494955 , -26.64786   ,  -9.3373375 ,
         12.1898985 , -10.893387  ,   1.9430412 , -11.230575  ,
        -20.572903  , -17.844627  ,   8.21945   ,   6.375868  ,
         -4.675418  ,   3.3220718 ,   0.73408353, -12.69471   ,
         -1.7531934 ,   0.93590397, -15.047206  , -12.389329  ,
        -14.8535185 ,   5.3839183 , -12.724849  , -11.156646  ,
          4.307355  ,   4.2647862 ]], dtype=float32)

In [207]:
np.savetxt("B.txt",img_fl,fmt="%f")